# Twitter datasets for water events

Author: Fadoua Ghourabi (fadouaghourabi@gmail.com)

Date: version @ July 17, 2019 

- **data**: dataframe to store extracted tweet texts and meta data (date, location, etc.) before processing
- **clean_tweets**: list to store tweet texts after nlp preprocessing (using function clean_collection)
- **data_clean_tweets**: dataframe copy of datasets data where tweet text is replaced by clean_tweets
- **data_clean_tweets_in_vocab**: a copy of data_clean_tweets where tweets that are not in the model vacabulary are dropped

To export for classification:
- **tweet_avg_w2v**: tweet data where vector representation is average vectors (gensim.models.word2vec)
- **tweet_avg_w2v_tfidf**: tweet data where vector representation is average vectors multiplied with the tfidf metric (gensim.models.word2vec and sklearn's TfidfVectorizer)
- **tweet_d2v**: twet data where vector representation is computed by gensim.models.doc2vec
- **tweet_avg_ft**: tweet data where vector representation uses fasttext corpora

In [1]:
import os
import time
import pandas as pd
import numpy as np
import gensim
#fasttext 0.9.1 
import fasttext
# clean_collection is a function that implements nlp preprocessing pipeline
from ipynb.fs.full.fr_twitter_water_nlp import clean_collection 
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# file contains manually labeled tweets
tw_path = "../datasets/twData_clean_labeled.csv"
# folder for corpus generated from collected tweets
corpusdir = 'corpus/'
# fast text coprpus (https://fasttext.cc/docs/en/crawl-vectors.html)
fast = '/Users/basho/fadouaproject/SafeWater/model/cc.fr.300.bin'
# folder for tweet datasets
dataset_path = "../datasets/"

In [3]:
# date of the most recent collection and location of extracted tweet
collection_date = "February 10, 2019 ~ July 20, 2019"
geo_loc = "Sfax (center), 400km (radius)"

### Data structure and NLP preprocessing

In [4]:
def load_water_tweets(desc=True):
    
    '''
    - Description: 
    load_water_tweets read the tweets and other meta data from the file by tw_path and
    returns a dataframe
    - Input:
    <desc> indicates wether to display description
    - Output:
    <tw_data> is a dataframe where columns are given by the header of the file tw_path
    - History:
    July 13, 2019 --> implementation, to fix: 11 columns should be 10 (extra "," in tw_path?)
    July 17, 2019 --> fixed: extra column removed in tw_path
    
    '''
    
    file = open(tw_path,"r")
    tw_data = pd.read_csv(file, header=0, encoding = 'utf-8') # utf-8 for better representation of french text
    
    if desc:
        print("=== Datasets for water-related tweets ===")
        print("Language: French")
        print("Collection date: {}".format(collection_date))
        print("Location: {}".format(geo_loc))
        print("Data size: {}".format(tw_data.shape))
        print("""Features: \n
              - Timestamp: date and time of collection. \n
              - TwDate: date and time of tweet publication. \n
              - TwLoc: localisation of user \n
              - TwUserName: user name\n
              - TwUserID: user's unique ID\n
              - TwContent: tweet message\n
              - ContentLoc: list of locations that are included in the tweet\n
              - urls: list of urls that are included in the tweet\n
              - Event: label --> water shortage (1) or not (0)""")
        print("=======================================")
    file.close()
    return tw_data

In [5]:
data = load_water_tweets()

=== Datasets for water-related tweets ===
Language: French
Collection date: February 10, 2019 ~ July 20, 2019
Location: Sfax (center), 400km (radius)
Data size: (748, 10)
Features: 

              - Timestamp: date and time of collection. 

              - TwDate: date and time of tweet publication. 

              - TwLoc: localisation of user 

              - TwUserName: user name

              - TwUserID: user's unique ID

              - TwContent: tweet message

              - ContentLoc: list of locations that are included in the tweet

              - urls: list of urls that are included in the tweet

              - Event: label --> water shortage (1) or not (0)


In [6]:
data.Event.isnull().values.any() # missing label?

False

In [7]:
data.Event.isna().values.any() # nan label?

False

In [8]:
data.TwContent.tail(10) # to check the coding of french text

738    Mon programme d’anniversaire est tombé à l’eau...
739    Envie de visiter la plus longue plage d'eau do...
740    #Réf H 418 - #Houmt_Souk #Djerba #Tunisie #Zon...
741    RT @Santedzmag: Arrivée à l'aéroport internati...
742    alors deux points importants à aborder sur ce ...
743    @nadhemtunisie C'est certain. Je fais pas touj...
744    Mon téléphone a pu de son, il est tombé dans l...
745    N'insultez pas le crocodile lorsque vos pieds ...
746    RT @Itiis_lauraine: tu fais nv on te demande s...
747    Genre là on fait nos ablutions avec de l'eau r...
Name: TwContent, dtype: object

In [9]:
tweets = data.TwContent # variable tweets contain raw tweet text as collected by the api
tweets.shape 

(748,)

In [10]:
def nlp_preprocessing(tws,lem=False):
    ''' 
    - Description:
    nlp_preprocessing simply calls clean_collection
    clean_collection applies nlp preprocessing (lower, remove urls, remove stopwords) on all the tweets
    - Input:
    tws: raw tweet text
    lem: lemmatization is optional, default False
    - Output:
    clean_tweets: prepreocessed tweet text
    duration: computation time
    - History:
    July 13, 2019 --> implementation, clean_collection is not prompt
    July 16, 2019 --> added assertion to make sure no tweet is lost
    '''
    start = time.time()
    clean_tweets = clean_collection(tws, lem) 
    end = time.time()
    duration = end - start # about 5s !
    
    assert len(clean_tweets) == tws.shape[0]
    
    return duration, clean_tweets

In [11]:
# preprocessing of tweets (see function clean_collection)
# warning: slow computation ~ >5s
start = time.time()
_, clean_tweets = nlp_preprocessing(tweets, True) # 
end = time.time()
len(clean_tweets), end - start

(748, 6.667438268661499)

In [12]:
def replace_column(data,column_index,newcolumn):
    '''
    - Description:
    replace_column replaces an entire column with new values, 
    e.g. the raw tweet text is replaced by its nlp preprocessing
    !! Attention !!
    if column_index is not an existing column then a new column is added
    - Input:
    data: original data
    column_index: a string indicating the column header
    new_column: new values for the column
    - Output:
    newdata: new data with new column values
    - History:
    July 13, 2019 --> implementation
    July 16, 2019 --> added assertions
    '''
    newdata = data.copy()
    assert not (id(newdata) == id(data))
    
    m = newdata.shape[0]
    newcolumn = list(newcolumn)
    newdata[column_index] = pd.Series(newcolumn)
    nm = len(newcolumn)
        
    assert (m == nm)
    # column is replaced or new column is added
    assert (newdata.shape == data.shape) or (newdata.shape[1] == data.shape[1]+1)
    
    return newdata

In [13]:
data_clean_tweets = replace_column(data,"TwContent",clean_tweets)

In [14]:
id(data_clean_tweets),id(data) # dataframes should not refer to the same object

(112757931200, 112757655480)

In [15]:
data_clean_tweets.shape

(748, 10)

### Make text corpus

We make a corpus out of the collected tweets. The corpus is used to generate word2vec representation (see next section). We make $n$ text files for $n$ tweets. All are savec in ``corpusdir``. 

For the time being, when adding new tweets, delete folder ``corpusdir`` and run ``make_text(clean_tweets)``. <font color='red'>This feature should be improved so that the exsisting corpus can be extended with new data.</font>

In [16]:
if not os.path.isdir(corpusdir):
    os.mkdir(corpusdir)

In [17]:
# one file - !! not used !!
def make_data_from_tweets(tweets):
    filename = 0
    file = open(corpusdir+'data.txt','a')
    for tw in tweets:
        file.write(tw)
        file.write('\n')
    file.close()

In [18]:
# seperate files
def make_text(data):
    '''
    - Description:
    make_text create corpus text files, one file for each tweet.
    - Input:
    tweets preferably nlp preprocessed tweets
    - Histroy:
    July 13, 2019 --> implementation, to fix: extending the corpus with new tweets
    '''
    
    filename = 0
    for text in data:
        filename+=1
        file = open(corpusdir+str(filename)+'.txt','w')
        file.write(text)#,encoding="UTF-8")
        file.close()

In [19]:
make_text(clean_tweets)

### tweet2vector representation (gensim)

According to NLTK dosucmentation on ``PlaintextCorpusReader``: "Reader for corpora that consist of plaintext documents. Paragraphs are assumed to be split using blank lines. Sentences and words can be tokenized using the default tokenizers, or by custom tokenizers specificed as parameters to the constructor."

However, our corpus already contains preprocessed tweets. We made this choice because of the language of the tweets requires non-common libraries. 

<font color='red'>For stronger credence, experiments should be performed: 
- ``PlaintextCorpusReader`` on raw corpora, 
- ``PlaintextCorpusReader`` on raw corpora + custom tokenizer</font>

In [20]:
newcorpus = PlaintextCorpusReader(corpusdir, '.*')

In [21]:
(clean_tweets[552],newcorpus.sents()[551])

('boire leau minéral tunisie cest faire tort propre santé planèt',
 ['rt',
  'mtvbasefranco',
  'cela',
  'vient',
  'togo',
  'fraiiiiiiiiiiiiiissss',
  'comme',
  'eau',
  'glacer',
  'donne',
  'jaloux',
  'dianuellaoff',
  'zagab'])

Gensim’s word2vec expects **a sequence of sentences as its input**. Each sentence is a list of words (utf8 strings). 

``class gensim.models.Word2Vec(sentences=None, corpus_file=None, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, iter=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=(), max_final_vocab=None)``

- Words that appear only once or twice in a corpus are probably uninteresting typos and garbage. In addition, there’s not enough data to make any meaningful training on those words, so the model ignores them. These words are not included in the vocabulary ``w2v_model.wv.vocab`` (OOV words). To change the minimum occurance of a word to be included in the vocabulary, use parameter ``min_count``.

- Word2Vec runs a neural network of default size 100. To change the size of NN, use parameter ``size``.

- Word2Vec uses a context window to limit the number of words in each context. To change the window size, use parameter ``window``

- The default algorithm is CBOW (parameter ``sg=0``). The use skipgram, set parameter ``sg=1``. 

<font color='red'>We should perform experiments with various values for the hyperparameters.</font>

In [22]:
# training Word2Vec on newcorpus sentenses
# CBOW algorithm, words occuring > 3 times make vocab, context window of size 3, NN of 50 layers 
w2v_model = gensim.models.Word2Vec(newcorpus.sents(), sg=0, min_count=3, window=5, size=50)

In [23]:
# size of the vocabulary (276 when min_count = 5)
len(w2v_model.wv.vocab)

753

In [24]:
w2v_model.wv["eau"]

array([-0.19761942,  0.1229533 , -0.06306787, -0.07563478,  0.09874278,
       -0.05666644,  0.03780708, -0.02814587,  0.01980024, -0.0251016 ,
       -0.06849147,  0.1328966 ,  0.0019268 ,  0.1127112 , -0.074561  ,
        0.12548004, -0.0458111 ,  0.14228219,  0.08896548,  0.05363914,
       -0.07233844,  0.06709651,  0.04879233,  0.05539123,  0.13120863,
        0.09444671, -0.05032353, -0.0623964 ,  0.06268214, -0.10559046,
        0.044142  , -0.1979832 ,  0.01949855,  0.01648371, -0.08589403,
       -0.04238088, -0.03303375,  0.06767984, -0.0140308 , -0.02164407,
       -0.02491418, -0.06206277, -0.03697249, -0.001055  , -0.0417621 ,
       -0.04004826, -0.05592086, -0.05017557, -0.01060602, -0.00343862],
      dtype=float32)

In [25]:
w2v_model.wv["coupure"].shape

(50,)

In [26]:
def similarity_two_words(model,w1, w2):
    sim = model.wv.similarity(w1,w2)
    #print("The similarity between <{}> and <{}>: ".format(w1, w2), sim)
    return sim

In [27]:
similarity_two_words(w2v_model, 'eau', 'potable')

0.99297035

In [28]:
similarity_two_words(w2v_model, 'eau', 'coupure')

0.97994286

In [29]:
similarity_two_words(w2v_model, 'eau', 'gafsa')

0.74831235

In [30]:
similarity_two_words(w2v_model, 'eau', 'tunis')

0.83031666

In [31]:
w2v_model.wv.most_similar('eau')

[('le', 0.9953315258026123),
 ('leau', 0.9950196146965027),
 ('avoir', 0.99333256483078),
 ('potable', 0.9929704070091248),
 ('tout', 0.9907903671264648),
 ('tunisie', 0.9903091788291931),
 ('cest', 0.9903061389923096),
 ('faire', 0.989640474319458),
 ('voir', 0.9894818663597107),
 ('deau', 0.9892990589141846)]

In [32]:
w2v_model.wv.most_similar('coupure')

[('faire', 0.9839985966682434),
 ('le', 0.9830204248428345),
 ('leau', 0.9807822704315186),
 ('eau', 0.9799428582191467),
 ('rt', 0.9793148636817932),
 ('cest', 0.9792321920394897),
 ('plus', 0.9788925051689148),
 ('potable', 0.9787129759788513),
 ('avoir', 0.9785737991333008),
 ('boir', 0.9758062362670898)]

In [33]:
w2v_model.wv.most_similar('distribution')

[('avoir', 0.9788521528244019),
 ('potable', 0.97800213098526),
 ('tunisie', 0.9765549898147583),
 ('leau', 0.9755337834358215),
 ('eau', 0.9744558930397034),
 ('projet', 0.9738720059394836),
 ('le', 0.9735943675041199),
 ('faire', 0.9734907150268555),
 ('deau', 0.9731074571609497),
 ('leur', 0.9725967049598694)]

#### Strategy 1: Averaging word2vec

How to compute the vector representation of a tweet from the vector representation of its words? One alternative is the sum the vector representation and divide by the lenght of the tweet.

In [34]:
def check_in_vocabulary(model,tws,vocab):
    '''
    - Description:
    check_in_vocabulary checks which words are not in the model's vocabulary. 
    These words cause errors as they cannot be converted to vectors.
    eg: KeyError "word 'satisfaction' not in vocabulary"
    - Input:
    model: the trained word2vector model
    tws: clean tweets
    vocab: the model's vocabulary. In case of gensim's Word2Vec, model.wv.vocab gives a dictionary of vocabulary.
    - Output:
    not_in_vocabulary: list of (row_index,tweet) where tweet is composed of words 
    that are not in the model's vocabulary.
    - History:
    July 16, 2019 --> implementation, we choose to ignore the words that are not in the vocabulary (OOV).
    '''
    not_in_vocabulary = []
    i = 0
    for tw in tws:
        n_words = 0
        for w in tw.split():
            if w in vocab: # careful! model.wv.vocab is not a complete list of unique words
                n_words += 1
        
        if n_words == 0: # meaning all the words in tw are not in model's vocabulary
            not_in_vocabulary.append((i,tw)) 
        i += 1
    
    return not_in_vocabulary

In [35]:
def drop_not_in_vocabulary(data,indexes):
    '''
    - Description:
    drop_not_in_vocabulary drops from data the tweets that are not in the model's vocabulary
    - Input:
    data: complete data with clean tweet texts, i.e. the dataset data_clean_tweets
    indexes: row indexes of tweets not in vocabulary
    - Output:
    data.drop(indexes) 
    '''
    return data.drop(indexes)

In [36]:
nv = check_in_vocabulary(w2v_model,clean_tweets,w2v_model.wv.vocab)
nv # empty nv means all tweets are composed of words in w2v_model.wv.vocab

[]

In [37]:
indexes = [i for (i, _) in nv]
data_clean_tweets_in_vocab = drop_not_in_vocabulary(data_clean_tweets,indexes)

In [38]:
data_clean_tweets_in_vocab.shape

(748, 10)

In [39]:
def tweets_to_avgvecs(model,tws,num_features):
    '''
    - Description:
    tweets_to_avgvecs computes the vector representation of a tweet. 
    To that end, it sums the vector representations of words, then divide by the nbr of words.
    Out of vocabulary words (OOV) are ignored.
    numpy arrays are used for efficient array arithmetics
    - Input:
    model: the trained model
    tws: clean tweets
    num_feature: the size of the vector representation
    - Output:
    vectors: numpy array of stacked vector representations of tweets
    not_in_vocabulary: tweets that are not in the vocabulary
    - History:
    July 16, 2019 --> implementation, 
    to fix: 
    a) one function for different models: e.g. w2v_avg, w2v_avg_tfidf, w2v_avg_ft
    b) what to do with OOV?
    '''
    vocab = model.wv.vocab # assuming gensim's word2vec model
    vectors = [] #np.empty((len(tweets), num_features)) # dtype='float32')
    not_in_vocabulary = []
    i = 0
    for tw in tws:
        tw_vec = np.zeros((num_features,), dtype='float32')
        n_words = 0
        
        for w in tw.split():
            if w in vocab: # careful! model.wv.vocab is not a complete list of unique words
                n_words += 1
                # summation of vectors of words in vocab
                # OOV are ignored
                tw_vec = np.add(tw_vec, model.wv[w]) 
                
        
        if (n_words > 0):
            tw_vec = np.divide(tw_vec, n_words)
            vectors.append(tw_vec.tolist()) #, axis=0)
        else:
            # not_in_vocabulary staures the tweets composed of words not in model.wv.vocab
            not_in_vocabulary.append((i,tw)) 
        i += 1
        
        assert len(vectors[0]) == num_features

    return (np.array(vectors),not_in_vocabulary)

In [40]:
size = w2v_model.vector_size
tws = data_clean_tweets_in_vocab.TwContent
tw_vectors, nv = tweets_to_avgvecs(w2v_model,tws,size)

In [41]:
tw_vectors.shape

(748, 50)

In [42]:
tweet_avg_w2v = replace_column(data_clean_tweets_in_vocab,"TwVec",list(tw_vectors))

In [43]:
tweet_avg_w2v.shape

(748, 11)

#### Strategy 2: averaging word2vec with TF-IDF

In [44]:
def tweets_to_avgvecs_with_tfidf(model,vectorizer,tws,num_features):
    '''
    - Description:
    tweets_to_avgvecs_with_tfidf computes the vector representation of a tweet. 
    To that end, it sums the vector representations of words multiplied by the tfidf representation,
    then divide by the nbr of words.
    Out of vocabulary words (OOV) are ignored.
    numpy arrays are used for efficient array arithmetics
    - Input:
    model: the trained model
    vectorizer: the tfidf model
    tws: clean tweets
    num_feature: the size of the vector representation
    - Output:
    vectors: numpy array of stacked vector representations of tweets
    not_in_vocabulary: tweets that are not in the vocabulary
    - History:
    July 16, 2019 --> implementation, 
    to fix: 
    a) one function for different models: e.g. w2v_avg, w2v_avg_tfidf, w2v_avg_ft
    b) what to do with OOV?
    '''
    tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
    vectors = [] #np.empty((len(tweets), num_features)) # dtype='float32')
    not_in_vocabulary = []
    i = 0
    for tw in tws:
        tw_vec = np.zeros((num_features,), dtype='float32')
        n_words = 0
        
        for w in tw.split():
            # careful! model.wv.vocab is not a complete list of unique words
            if (w in model.wv.vocab) and (w in vectorizer.vocabulary_): 
                n_words += 1
                vec = model.wv[w]*tfidf[w]
                tw_vec = np.add(tw_vec, vec)
                
        
        if (n_words > 0):
            tw_vec = np.divide(tw_vec, n_words)
            vectors.append(tw_vec.tolist()) #, axis=0)
        else:
            # not_in_vocabulary staures the tweets composed of words not in model.wv.vocab
            not_in_vocabulary.append((i,tw)) 
        i += 1

    return (np.array(vectors),not_in_vocabulary)

In [45]:
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(newcorpus.words())

<10210x3029 sparse matrix of type '<class 'numpy.float64'>'
	with 10145 stored elements in Compressed Sparse Row format>

In [46]:
vectorizer.idf_.shape

(3029,)

In [47]:
size = w2v_model.vector_size
tws = data_clean_tweets_in_vocab.TwContent
tw_vectors, nv = tweets_to_avgvecs_with_tfidf(w2v_model,vectorizer,tws,size)

In [48]:
tw_vectors.shape

(748, 50)

In [49]:
tweet_avg_w2v_tfidf = replace_column(data_clean_tweets_in_vocab,"TwVec",list(tw_vectors))
tweet_avg_w2v_tfidf.shape

(748, 11)

In [50]:
#tweet_avg_w2v_tfidf

#### Strategy 3: doc2vec

In [51]:
def tag_tweets(tws):
    '''
    - Description:
    tag_tweets generates tweets along with iterable tags, 
    which are the input document format needed by Doc2Vec
    - Input:
    tws: a list of tweet where each tweet is tokenized (composed of a list of words)
    - Output:
    docs: tagged tweets
    - History:
    July 17, 2019 --> implementation
    '''
    docs = []
    i = 0
    for tw in tws:
        document = TaggedDocument(tw, ["t"+str(i)])
        docs.append(document)
        i += 1
    return docs

In [52]:
# we make tagged tweets
docs = tag_tweets(newcorpus.sents())

In [53]:
# d2v vector size is 300, context window 3, min occurrence is 3, epochs for computation is 20
d2v = gensim.models.Doc2Vec(vector_size=300, window=3, min_count=3, epochs=20)
d2v.build_vocab(docs)

In [54]:
# d2v model is trainted over the entire corpus (size is d2v.corpus_count)
# field epochs is required, otherwise error
d2v.train(docs, total_examples=d2v.corpus_count, epochs=d2v.iter)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
# to obtain vector representation of a sentence:
# first tokenize the sentence
# pass the list of words to function infer_vector
#d2v.infer_vector(["coupure","eau","gafsa"])

In [56]:
# d2v saves the vector representation in d2v.docvecs
# we transform the vectors to a list for easy manipulation
tw_dic = dict(zip(d2v.docvecs.doctags, d2v.docvecs))
vectors = []
for _,vec in tw_dic.items():
    vectors.append(vec)

In [57]:
tweet_d2v= replace_column(data_clean_tweets_in_vocab,"TwVec", vectors)
tweet_d2v.shape

(748, 11)

In [58]:
#tweet_avg_d2v.head()

### tweet2vec representation (fasttext corpus)

In [59]:
def check_in_vocabulary_ft(model,tws):
    '''
    - Description:
    check_in_vocabulary_ft checks which words are not in the model's vocabulary. 
    These words cause errors as they cannot be converted to vectors.
    - Input:
    model: the trained fasttext model
    tws: clean tweets
    - Output:
    not_in_vocabulary: list of (row_index,tweet) where tweet is composed of words 
    that are not in the model's vocabulary.
    - History:
    July 16, 2019 --> implementation, we choose to ignore the words that are not in the vocabulary (OOV).
    to do: merge with functioncheck_in_vocabulary
    '''
    vocab = model.get_words()
    not_in_vocabulary = []
    i = 0
    for tw in tws:
        n_words = 0
        for w in tw.split():
            if w in vocab: 
                n_words += 1
        
        if n_words == 0:
            not_in_vocabulary.append((i,tw)) 
        i += 1
    
    return not_in_vocabulary

In [60]:
def tweets_to_ft_avgvecs(model,tws,num_features):
    '''
    - Description:
    tweets_to_ft_avgvecs computes the vector representation of a tweet based on fasttext corporus for french. 
    To that end, it sums the vector representations of words, then divide by the nbr of words.
    Out of vocabulary words (OOV) are ignored.
    numpy arrays are used for efficient array arithmetics
    - Input:
    model: the trained model
    tws: clean tweets
    num_feature: the size of the vector representation
    - Output:
    vectors: numpy array of stacked vector representations of tweets
    not_in_vocabulary: tweets that are not in the vocabulary
    - History:
    July 16, 2019 --> implementation, 
    to fix: 
    a) one function for different models: e.g. w2v_avg, w2v_avg_tfidf, w2v_avg_ft
    b) what to do with OOV?
    '''
    vocab = model.get_words()
    vectors = [] #np.empty((len(tweets), num_features)) # dtype='float32')
    not_in_vocabulary = []
    i = 0
    for tw in tws:
        tw_vec = np.zeros((num_features,), dtype='float32')
        n_words = 0
        
        for w in tw.split():
            if w in vocab: 
                n_words += 1
                tw_vec = np.add(tw_vec, model[w])
                
        
        if (n_words > 0):
            tw_vec = np.divide(tw_vec, n_words)
            vectors.append(tw_vec.tolist()) #, axis=0)
        else:
            # not_in_vocabulary staures the tweets composed of words not in model.wv.vocab
            not_in_vocabulary.append((i,tw)) 
        i += 1

    return (np.array(vectors),not_in_vocabulary)

In [61]:
# we load the corpus indicated by the file fast
# warning: slow computation ~ 30s (sometimes 19s)
start = time.time()
fasttext_model = fasttext.load_model(fast)
end = time.time()
end - start

25.685548067092896

In [62]:
# size of the vocabulary - obviously larger than water corpus
len(fasttext_model.get_words())

2000000

In [63]:
# dimention of vector representation
fasttext_model["eau"].shape

(300,)

In [64]:
# tweets that are not in the vocabulary, i.e. composed of OOV only
# warning: slow computation ~ 34s 
start = time.time()
nv = check_in_vocabulary_ft(fasttext_model,clean_tweets)
end = time.time()
end - start

48.23744297027588

In [65]:
indexes = [i for (i, _) in nv]
data_clean_tweets_in_vocab = drop_not_in_vocabulary(data_clean_tweets,indexes)

In [66]:
data_clean_tweets_in_vocab.shape

(748, 10)

In [67]:
# computing vector representation of tweets (using fasttext model)
# warning: slow computation ~ 32s
tws = data_clean_tweets_in_vocab.TwContent
start = time.time()
tw_vectors, nv = tweets_to_ft_avgvecs(fasttext_model,tws,300)
end = time.time()
end - start

42.987359046936035

In [68]:
tweet_avg_ft = replace_column(data_clean_tweets_in_vocab,"TwVec",list(tw_vectors))
tweet_avg_w2v_tfidf.shape

(748, 11)